# Scan in $(hkl)$ coordinates

This notebook demonstrates how to scan in $(hkl)$ coordinates. It uses the
simulated 4-circle geometry from the `"hkl_soleil"` solver. The wavelength and
sample are whatever the simulator provides as defaults.

## Setup

First, create the simulated 4-circle diffractometer object (`e4cv`).

In [1]:
from hklpy2 import SimulatedE4CV

e4cv = SimulatedE4CV("", name="e4cv")

Setup Bluesky for running the scans with the `RE` object.  The `bec` object will
show a table of the data collected for each scan.  

For this simple demonstration, we won't add a databroker catalog.

In [2]:
from bluesky import RunEngine, plans as bp
from bluesky.callbacks.best_effort import BestEffortCallback

bec = BestEffortCallback()
RE = RunEngine()
RE.subscribe(bec)
bec.disable_plots()

We'll **import a simulator** (ready to use) from the `ophyd` package as a noisy detector.

In [3]:
from ophyd.sim import noisy_det

## (h10) scan

Scan the (reciprocal space) $h$ axis from -0.5 to +0.5 with $k=1$ and $l=0$.
This is called an $(h10)$ scan.

<details>

The computation to convert reciprocal-space values $(h,k,l)$ into real-space
angles ($\omega$, $\chi$, $\phi$, $2\theta$) is called the `forward()`
transformation.  The transformation is not necessarily unique.  The most common
way to reduce the number of *solutions* is to tell the solver which `mode` to
use.  The `mode` adds an additional pre-designed rule that constrains the
acceptable solutions.  The solver's geometry (in this case `E4CV`) provides the
list of known modes.

Note: Even with a chosen mode, the solution might not be unique.  In such cases,
the first solution returned by the `forward()` transformation is chosen.  The
user can change this by providing a different function for the diffractometer's
`_forward_solution` attribute.  The default is the
`hklpy2.diffract.pick_first_item()` function.

</details>

Here, the diffractometer starts with `"bissector"` mode (requires `tth = 2*omega`).

In [4]:
print(f"{e4cv.operator.solver.mode=!r}")
e4cv.k.move(1)
e4cv.l.move(0)
RE(bp.scan([noisy_det], e4cv.h, -0.5, 0.5, 11))

e4cv.operator.solver.mode='bissector'


Transient Scan ID: 1     Time: 2024-06-18 10:21:11
Persistent Unique Scan ID: '83362fea-6cf4-4837-859f-b9cf0317a581'
New stream: 'primary'
+-----------+------------+------------+------------+
|   seq_num |       time |     e4cv_h |  noisy_det |
+-----------+------------+------------+------------+
|         1 | 10:21:11.2 |     -0.500 |      0.914 |
|         2 | 10:21:11.2 |     -0.400 |      1.032 |
|         3 | 10:21:11.2 |     -0.300 |      1.058 |
|         4 | 10:21:11.3 |     -0.200 |      1.055 |
|         5 | 10:21:11.3 |     -0.100 |      1.006 |
|         6 | 10:21:11.3 |      0.000 |      1.035 |
|         7 | 10:21:11.3 |      0.100 |      1.007 |
|         8 | 10:21:11.3 |      0.200 |      1.018 |
|         9 | 10:21:11.3 |      0.300 |      1.062 |
|        10 | 10:21:11.3 |      0.400 |      1.060 |
|        11 | 10:21:11.3 |      0.500 |      1.023 |
+-----------+------------+------------+------------+
generator scan ['83362fea'

('83362fea-6cf4-4837-859f-b9cf0317a581',)

**Clearly we see** that $h$ has been stepped across the range of -0.5 to +0.5.
Values for the noisy detector have been reported at each step.  But we want to
know about *all* the $hkl$ and angle values so we can observe the effects of
`"bissector"` mode.

### Scan again, showing all $(hkl)$ and real-space axes

Repeat the scan, same as before with a slight variation.  This time, add the
`e4cv` object as an additional detector.  

Here, we add `e4cv` *before* `noisy_det` so all diffractometer axes are listed
first in the table.

In [5]:
print(f"{e4cv.operator.solver.mode=!r}")
e4cv.k.move(1)
e4cv.l.move(0)
RE(bp.scan([e4cv, noisy_det], e4cv.h, -0.5, 0.5, 11))

e4cv.operator.solver.mode='bissector'


Transient Scan ID: 2     Time: 2024-06-18 10:21:11
Persistent Unique Scan ID: 'c46db789-a333-4268-9b64-5362c1884274'
New stream: 'primary'
+-----------+------------+------------+------------+------------+------------+------------+------------+------------+------------+
|   seq_num |       time |     e4cv_h |     e4cv_k |     e4cv_l | e4cv_omega |   e4cv_chi |   e4cv_phi |   e4cv_tth |  noisy_det |
+-----------+------------+------------+------------+------------+------------+------------+------------+------------+------------+
|         1 | 10:21:11.5 |     -0.500 |      1.000 |      0.000 |    -33.988 |    -63.435 |     90.000 |    -67.976 |      0.985 |
|         2 | 10:21:11.5 |     -0.400 |      1.000 |      0.000 |    -32.583 |    -68.199 |     90.000 |    -65.165 |      0.955 |
|         3 | 10:21:11.5 |     -0.300 |      1.000 |     -0.000 |    -31.468 |    -73.301 |     90.000 |    -62.935 |      1.059 |
|         4 | 10:21:11.5 |     -0.2

('c46db789-a333-4268-9b64-5362c1884274',)

## What other modes are available?

In [6]:
e4cv.operator.solver.modes

['bissector',
 'constant_omega',
 'constant_chi',
 'constant_phi',
 'double_diffraction',
 'psi_constant']

## Scan $(h10)$ holding $\omega$ at -30 degrees

Set the mode to `"constant_omega"`, then set $\omega=-30$ degrees.

In [7]:
e4cv.operator.solver.mode = "constant_omega"
e4cv.omega.move(-30)
print(f"{e4cv.omega.position=!r}")

e4cv.omega.position=-30


**Run the scan again** with the same command.

In [8]:
print(f"{e4cv.operator.solver.mode=!r}")
e4cv.k.move(1)
e4cv.l.move(0)
RE(bp.scan([e4cv, noisy_det], e4cv.h, -0.5, 0.5, 11))

e4cv.operator.solver.mode='constant_omega'


Transient Scan ID: 3     Time: 2024-06-18 10:21:11
Persistent Unique Scan ID: 'babc7b30-4d9d-4a28-b2fa-56e4793d3281'
New stream: 'primary'
+-----------+------------+------------+------------+------------+------------+------------+------------+------------+------------+
|   seq_num |       time |     e4cv_h |     e4cv_k |     e4cv_l | e4cv_omega |   e4cv_chi |   e4cv_phi |   e4cv_tth |  noisy_det |
+-----------+------------+------------+------------+------------+------------+------------+------------+------------+------------+
|         1 | 10:21:11.8 |     -0.500 |      1.000 |      0.000 |    -30.000 |    -63.714 |     81.054 |    -67.976 |      1.008 |
|         2 | 10:21:11.8 |     -0.400 |      1.000 |     -0.000 |    -30.000 |    -68.345 |     83.031 |    -65.165 |      1.072 |
|         3 | 10:21:11.8 |     -0.300 |      1.000 |     -0.000 |    -30.000 |    -73.364 |     84.887 |    -62.935 |      1.048 |
|         4 | 10:21:11.8 |    

('babc7b30-4d9d-4a28-b2fa-56e4793d3281',)